# Master test notebook to run all user notebooks
    

### Author: AWS Professional Services Emerging Technology and Intelligent Platforms Group
### Date: Feb 4 2021

In [1]:
import os
import sys
import boto3
from aws_orbit_sdk import controller
from aws_orbit_sdk.common import get_workspace,get_scratch_database
from botocore.exceptions import ClientError
s3 = boto3.client('s3')
sns = boto3.client('sns')
ssm = boto3.client('ssm')

In [2]:
workspace = get_workspace()
scratch_glue_db = get_scratch_database()
team_space = workspace['team_space']
# DO NOT RUN THIS NOTEBOOK IN LAKE CREATOR TEAM SPACE 
assert team_space == 'lake-user'
scratch_bucket = workspace['ScratchBucket']
(team_space, scratch_bucket, )

('lake-user',
 's3://orbit-foundation-public2-scratch-495869084367-cpelwk/lake-user')

In [3]:
def checkNotebooks(executions, expected_count):
    assert len(executions) == expected_count
    for index, row in executions.iterrows():
        if 'error@' in row['relativePath']:
            raise AssertionError('error in ' + row['relativePath'])
    print("SUCCESS")

### Cleanup

In [4]:
!find /home/jovyan/shared/regression/notebooks/B-DataAnalyst -type f -exec rm -fR {} \;
!find /home/jovyan/shared/regression/notebooks/I-Image -type f -exec rm -fR {} \;
!find /home/jovyan/shared/regression/notebooks/H-Model-Development -type f -exec rm -fR {} \;
!rm -f /home/jovyan/shared/regression/PASSED

find: ‘/home/jovyan/shared/regression/notebooks/I-Image’: No such file or directory
find: ‘/home/jovyan/shared/regression/notebooks/H-Model-Development’: No such file or directory


### Configure regression run

In [5]:
%%bash --out output --err error

ls ../B-DataAnalyst/*.ipynb 
ls ../I-Image/*.ipynb
ls ../H-Model-Development/*.ipynb

In [6]:
output

'../B-DataAnalyst/Example-1-SQL-Analysis-Athena.ipynb\n../B-DataAnalyst/Example-2-SQL-Analysis-RedshiftSpectrum.ipynb\n../B-DataAnalyst/Example-3-Spark-EMR-on-EKS.ipynb\n../B-DataAnalyst/Example-4-Glue-Databrew.ipynb\n../B-DataAnalyst/Example-6-Schedule-Notebook.ipynb\n../B-DataAnalyst/Example-8-SDK-Controller-Sched.ipynb\n../B-DataAnalyst/Example-90-Failure-Behavior.ipynb\n../I-Image/Example-1-simple.ipynb\n../I-Image/Example-2-spark.ipynb\n../I-Image/Example-3-gpu.ipynb\n../H-Model-Development/Example-1-SageMaker-xgboost_mnist.ipynb\n../H-Model-Development/Example-2-SageMaker-Batch Transform - breast cancer prediction with high level SDK.ipynb\n../H-Model-Development/Example-3-Ray Job Example.ipynb\n../H-Model-Development/Example-4-Ray Tune Example.ipynb\n'

In [7]:
notebooks_run_config = {
    # a list of notebooks names to skip the execution for. Example: ["Example-7-Data-Profiling"]
    "black_list": ['Example-3-Spark-EMR-on-EKS','Example-3-Ray Job Example','Example-4-Ray Tune Example'],
    "white_list": [],          # if not empty, only those will run. Example: ["Example-7-Data-Profiling"]
    "optional_list": [],       # indicates to ignoore a failure. Example: ["Example-6-Schedule-Notebook", "Example-8-LakeFormation-Security"]
    "minimum_successful": 1,   # number of minimum notebooks to be completed to consider entire test not failed (has an effect when this number is larger than number of mandatory )
    "maxRetries": 3,           # max number of attempts to execute a notebook
    "notebooks_to_run": [],     # all noootebooks for execution.
    "sagemaker_notebooks_list": ["Example-1-xgboost_mnist", "Example-4-Batch Transform - breast cancer prediction with high level SDK"] #sagemaker notebooks with small profile
}
 
for p in output.split('\n'):
    if (len(p)<2):
        continue 
    parts = p.split('/')
    nb_name, nb_folder = parts[2][::-1].split('.',1)[1][::-1], parts[1]
    if nb_name in notebooks_run_config["black_list"]:
        # ignore white list. black list is having highest priority for filters
        continue
    if not notebooks_run_config["white_list"] or nb_name in notebooks_run_config["white_list"]:
        # run notebook if white list is empty or if the notebook is in white list.
        if nb_folder in ["H-Model-Development"]:
            notebooks_run_config["notebooks_to_run"].append({"folder": nb_folder, "name": nb_name, "profile": "small"})
        else:
            notebooks_run_config["notebooks_to_run"].append({"folder": nb_folder, "name": nb_name})

notebooks_run_config

{'black_list': [],
 'white_list': [],
 'optional_list': [],
 'minimum_successful': 1,
 'maxRetries': 3,
 'notebooks_to_run': [{'folder': 'B-DataAnalyst',
   'name': 'Example-1-SQL-Analysis-Athena'},
  {'folder': 'B-DataAnalyst',
   'name': 'Example-2-SQL-Analysis-RedshiftSpectrum'},
  {'folder': 'B-DataAnalyst', 'name': 'Example-3-Spark-EMR-on-EKS'},
  {'folder': 'B-DataAnalyst', 'name': 'Example-4-Glue-Databrew'},
  {'folder': 'B-DataAnalyst', 'name': 'Example-6-Schedule-Notebook'},
  {'folder': 'B-DataAnalyst', 'name': 'Example-8-SDK-Controller-Sched'},
  {'folder': 'B-DataAnalyst', 'name': 'Example-90-Failure-Behavior'},
  {'folder': 'I-Image', 'name': 'Example-1-simple'},
  {'folder': 'I-Image', 'name': 'Example-2-spark'},
  {'folder': 'I-Image', 'name': 'Example-3-gpu'},
  {'folder': 'H-Model-Development',
   'name': 'Example-1-SageMaker-xgboost_mnist'},
  {'folder': 'H-Model-Development',
   'name': 'Example-2-SageMaker-Batch Transform - breast cancer prediction with high level S

In [8]:
# New implementation of the cell below
import time

def start_notebooks(run_config):
    _containers = []
    for nb in run_config["notebooks_to_run"]:
        notebook_to_run = {
            "tasks": [{
                      "notebookName": "{}.ipynb".format(nb['name']),
                      "sourcePath": f"shared/samples/notebooks/{nb['folder']}",
                      "targetPath": "shared/regression/notebooks/{}".format(nb['folder']),
                      "params": {
                      }
                    }],
            "compute": {
              "container" : {
                  "p_concurrent": "1",
                  "profile": "small"
              },
              "node_type": "ec2",
              "profile": "small"
            },
        }
        if "profile" in nb:
            notebook_to_run["compute"]["profile"] = nb["profile"]

        container = controller.run_notebooks(notebook_to_run)
        print("notebookName: " + str(container))
        _containers.append(container)
    return _containers


def update_run_config(run_config, execution_results):
    executed = run_config['notebooks_to_run']
    run_config['notebooks_to_run'] = [] #reset notebooks for the next execution
    
    # if nothing failed
    if not execution_results['failed']:
        return run_config
    
    for nb in executed:
        if nb['name'] in execution_results['failed']:
            run_config['notebooks_to_run'].append(nb)
    
    return run_config

In [9]:
def get_execution_results(run_config):
    result = {"success": [], "failed": []}
    for test in run_config['notebooks_to_run']:
        executions = controller.get_execution_history(f"shared/regression/notebooks/{test['folder']}", f"{test['name']}.ipynb") 
        nb_name = test['name']
        failed = False
        for index, row in executions.iterrows():
            res = row.get('relativePath')
            nb_name,folder_name  = (os.path.basename(os.path.dirname(res)), 
                                    os.path.dirname(res))
            if 'error@' in res:
                if 'Failure-Behavior' not in folder_name:
                    failed = True
            else:
                if 'Failure-Behavior'  in folder_name:
                    failed = True    
        if failed:
            result["failed"].append({"folder": folder_name, "name": nb_name})
        else:
            result["success"].append({"folder": folder_name, "name": nb_name})
    return result    

In [10]:
notebooks_run_config["maxRetries"] = 2

In [ ]:
%%time
success = False
attempt = 0
run_config = notebooks_run_config
containers = []
while(attempt < notebooks_run_config["maxRetries"] and not success):
    attempt += 1
    print(f"Starting notebooks. Attempt {attempt}. Run config: {run_config}")
    containers = start_notebooks(run_config)
    controller.wait_for_tasks_to_complete(containers, 120,45, False)
    results = get_execution_results(run_config)
    print(f'Attemp {attempt} finished. Results: {results}')
    run_config = update_run_config(run_config, results)
    success = not run_config["notebooks_to_run"]


Starting notebooks. Attempt 1. Run config: {'black_list': [], 'white_list': [], 'optional_list': [], 'minimum_successful': 1, 'maxRetries': 2, 'notebooks_to_run': [{'folder': 'B-DataAnalyst', 'name': 'Example-1-SQL-Analysis-Athena'}, {'folder': 'B-DataAnalyst', 'name': 'Example-2-SQL-Analysis-RedshiftSpectrum'}, {'folder': 'B-DataAnalyst', 'name': 'Example-3-Spark-EMR-on-EKS'}, {'folder': 'B-DataAnalyst', 'name': 'Example-4-Glue-Databrew'}, {'folder': 'B-DataAnalyst', 'name': 'Example-6-Schedule-Notebook'}, {'folder': 'B-DataAnalyst', 'name': 'Example-8-SDK-Controller-Sched'}, {'folder': 'B-DataAnalyst', 'name': 'Example-90-Failure-Behavior'}, {'folder': 'I-Image', 'name': 'Example-1-simple'}, {'folder': 'I-Image', 'name': 'Example-2-spark'}, {'folder': 'I-Image', 'name': 'Example-3-gpu'}, {'folder': 'H-Model-Development', 'name': 'Example-1-SageMaker-xgboost_mnist'}, {'folder': 'H-Model-Development', 'name': 'Example-2-SageMaker-Batch Transform - breast cancer prediction with high lev

INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[]
INFO:root:volume_mounts:[]


notebookName: {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-hhbv6', 'NodeType': 'ec2', 'tasks': [{'notebookName': 'Example-1-SQL-Analysis-Athena.ipynb', 'sourcePath': 'shared/samples/notebooks/B-DataAnalyst', 'targetPath': 'shared/regression/notebooks/B-DataAnalyst', 'params': {}}]}


INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[]
INFO:root:volume_mounts:[]


notebookName: {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-rwdfg', 'NodeType': 'ec2', 'tasks': [{'notebookName': 'Example-2-SQL-Analysis-RedshiftSpectrum.ipynb', 'sourcePath': 'shared/samples/notebooks/B-DataAnalyst', 'targetPath': 'shared/regression/notebooks/B-DataAnalyst', 'params': {}}]}


...

INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[]
INFO:root:volume_mounts:[]


notebookName: {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-fhx8f', 'NodeType': 'ec2', 'tasks': [{'notebookName': 'Example-3-Ray Job Example.ipynb', 'sourcePath': 'shared/samples/notebooks/H-Model-Development', 'targetPath': 'shared/regression/notebooks/H-Model-Development', 'params': {}}]}


INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[]
INFO:root:volume_mounts:[]
INFO:root:Waiting for 14 tasks [{'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-hhbv6', 'NodeType': 'ec2', 'tasks': [{'notebookName': 'Example-1-SQL-Analysis-Athena.ipynb', 'sourcePath': 'shared/samples/notebooks/B-DataAnalyst', 'targetPath': 'shared/regression/notebooks/B-DataAnalyst', 'params': {}}]}, {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-rwdfg', 'NodeType': 'ec2', 'tasks': [{'notebookName': 'Example-2-SQL-Analysis-RedshiftSpectrum.ipynb', 'sourcePath': 'shared/samples/notebooks/B-DataAnalyst', 'targetPath': 'shared/regression/notebooks/B-DataAnalyst', 'params': {}}]}, {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-lsbt9', 'NodeType': 'ec2', 'ta

notebookName: {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-c6gqz', 'NodeType': 'ec2', 'tasks': [{'notebookName': 'Example-4-Ray Tune Example.ipynb', 'sourcePath': 'shared/samples/notebooks/H-Model-Development', 'targetPath': 'shared/regression/notebooks/H-Model-Development', 'params': {}}]}


INFO:root:Task {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-6nbfm', 'NodeType': 'ec2', 'tasks': [{'notebookName': 'Example-90-Failure-Behavior.ipynb', 'sourcePath': 'shared/samples/notebooks/B-DataAnalyst', 'targetPath': 'shared/regression/notebooks/B-DataAnalyst', 'params': {}}]} is running with status {'active': 1,
 'completion_time': None,
 'conditions': None,
 'failed': None,
 'start_time': datetime.datetime(2021, 5, 20, 12, 39, 15, tzinfo=tzlocal()),
 'succeeded': None}
INFO:root:Task {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ec2-runner-k6jxx', 'NodeType': 'ec2', 'tasks': [{'notebookName': 'Example-1-simple.ipynb', 'sourcePath': 'shared/samples/notebooks/I-Image', 'targetPath': 'shared/regression/notebooks/I-Image', 'params': {}}]} is running with status {'active': 1,
 'completion_time': None,
 'conditions': None,
 'failed': None,
 'start_time': datetime.datetime(2021, 5, 20, 12, 39, 17, tzinfo=tzlocal()),
 'succeeded': None}
INFO:root:Task {'Ex

In [ ]:
%%time
controller.wait_for_tasks_to_complete(containers, 120,45, False)

In [ ]:
results

In [ ]:
if len(results['failed']) > 0:
    topicArn = ssm.get_parameter(Name="Orbit-Slack-Notifications")["Parameter"]["Value"]
    unmar_topicArn = json.loads(topicArn)
    try:
        response = sns.get_topic_attributes(
            TopicArn=unmar_topicArn['TopicArn']
        )
        d={}
        d['message']=results['failed']
        response = sns.publish(
            TopicArn=str(unmar_topicArn['TopicArn']),
            Message=json.dumps({'default': json.dumps(d)}),
            Subject='string',
            MessageStructure='json'
        )
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NotFoundException':
            print ("SNS Topic doesnt exist")

assert len(results['failed']) == 0

In [ ]:
!echo "PASSED" >> /home/jovyan/shared/regression/PASSED
!ls /home/jovyan/shared/regression/
!sleep 15s

# End of notebook
